In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')



# Any results you write to the current directory are saved as output.

# Read data

In [ ]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_gender = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
values = df_train.values

df_train.isnull().sum()

In [ ]:
df_train

what should we do ?
2224 were on ship, 1502 were dead, so 722 were survived.
891 are on train set, which is ground truth.
we should predict on test set.

# survived for each feature

In [ ]:
df_train.groupby(['Sex', 'Survived']).Sex.count()

In [ ]:
df_train.groupby(['Age', 'Survived']).Age.count()

In [ ]:
df_train.groupby(['SibSp', 'Survived']).SibSp.count()

In [ ]:
df_train.groupby(['Parch', 'Survived']).Parch.count()

In [ ]:
df_train.groupby(['Fare', 'Survived']).Fare.count()

In [ ]:
df_train.groupby(['Cabin', 'Survived']).Cabin.count()

In [ ]:
df_train.groupby(['Embarked', 'Survived']).Embarked.count()

# visualization by number of survived

In [ ]:
df_train.groupby(['Sex','Survived']).Survived.count().unstack('Sex').plot.bar()


In [ ]:
df_train.groupby(['SibSp','Survived']).Survived.count().unstack('SibSp').plot.bar()

In [ ]:
df_train.groupby(['Parch','Survived']).Survived.count().unstack('Parch').plot.bar()

In [ ]:
df_train.groupby(['Embarked','Survived']).Survived.count().unstack('Embarked').plot.bar()

In [ ]:
df_train_new = df_train.copy()


In [ ]:
df_train.groupby('Survived').Fare.hist()

# how much percent survived?

In [ ]:
df_train['Survived'].mean()

Social classes were heavily stratified in the early twentieth century. This was especially true on the Titanic, where the luxurious first-class areas were completely off limits to the middle-class passengers in second class, and especially to those who carried a third class “economy price” ticket. Pclass column shows Ticket class.
Pclass : A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

In [ ]:
df_train.groupby(['Pclass']).mean()

it means that on ticket class 1, which it means the rich people were there, have 62% chance to survived.
on ticket class 2, which middle-class passengers were there,have 47% chance to survived.
on ticket class 3, which poor passengers were there, have 24% chance to survived.
in conclusion, most of survived people were from rich people.

# Average by Pclass and Sex

In [ ]:
df_pclass_sex = df_train.groupby(['Pclass','Sex']).mean()
df_pclass_sex

In [ ]:
df_pclass_sex = df_pclass_sex['Survived']
df_pclass_sex

In [ ]:
df_pclass_sex.plot.bar()

so, it is so fantastic. 
from these dataframe and plot, we can understand female has more chance to survive. even in poor or rich o middle-class, female has more chance.
Because in that time, when people were allowed to be in lifeboat, women and children have priority to be in there. so more women survived.

#  To check Age 

In [ ]:
df_train.describe()

we see that min and max in age is 0.4 and 80.

In [ ]:
df_age = pd.cut(df_train['Age'], np.arange(0, 90, 10))
df_mean_age = df_train.groupby(df_age).mean()
df_mean_age['Survived'].plot.bar()

In [ ]:
df_age = pd.cut(df_train['Age'], np.arange(0, 90, 5))
df_mean_age = df_train.groupby(df_age).mean()
df_mean_age['Survived'].plot.bar()

# Go for preparing data for machine learning

we have some issues. 
Sex, Cabin and Embarked are categorical.
Age, Cabin and Embarked have null cells.

In [ ]:
df_train.count()

**categorical features** convert Sex and Embarked columns to numeric values


Embarked has 2 null cell, so i drop 2 rows.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
df_train['Sex_code'] = label.fit_transform(df_train['Sex']) #male 1 female 0 
df_train_n = df_train.drop(['Sex' , 'Cabin'] , axis=1)

df_train_n

In [ ]:
df_train_n['Embarked'].isnull().sum()

In [ ]:
df_train_n = df_train_n.dropna(subset = ['Embarked'])
df_train_n

now we drop null rows in Embarked column. so we change categorical to numeric.

In [ ]:
df_train_n['Embarked_code'] = label.fit_transform(df_train_n['Embarked']) #s 2 c 0 q 1
df_train_n = df_train_n.drop(['Embarked'] , axis=1)
df_train_n

and now we drop name and ticket

In [ ]:
df_train_n = df_train_n.drop(['Name', 'Ticket'] , axis=1)
df_train_n

split x , y 

In [ ]:
y_train = df_train_n['Survived'].values
y_train

In [ ]:
col = list(df_train_n.columns.values)
col.remove('Survived')
col

In [ ]:
x_train = df_train_n[col].values
x_train

**preparing test set**

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.info()

In [ ]:
#

df_test['Sex_code'] = label.fit_transform(df_test['Sex']) #male 1 female 0 
df_test['Embarked_code'] = label.fit_transform(df_test['Embarked']) #s 2 c 0 q 1
#df_test = df_test.drop(['Sex' , 'Cabin' , 'Name' , 'Ticket', 'Embarked'] , axis=1)
df_test

In [ ]:
col_test = ['PassengerId', 'Pclass' , 'Age' , 'SibSp' , 'Parch', 'Fare' , 'Sex_code', 'Embarked_code']
x_test = df_test[col_test].values
x_test

In [ ]:
y_test = df_gender['Survived'].values
y_test

**Go for XGboost **

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
data_dmatrix = xgb.DMatrix(data=x_train,label=y_train)
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(x_train,y_train)

preds = xg_reg.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

#  ****Correlation between features******

In [ ]:
plt.figure(figsize = (12,10))
cor = df_train.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()